# 🎓 RIS PhD Ultimate Research Dashboard

## Comprehensive, Fully Customizable Research Platform

**Version:** 1.0.0  
**Purpose:** Professional-grade research platform for RIS probe-based ML experiments

### Features:
- ✅ 5 Customization Tabs (System, Model, Training, Evaluation, Visualization)
- ✅ 19 Pre-defined Model Architectures + Custom
- ✅ 6 Probe Types (continuous, binary, 2bit, hadamard, sobol, halton)
- ✅ 25+ Plot Types
- ✅ Multi-Model Comparison
- ✅ Multi-Seed Statistical Analysis
- ✅ Config Save/Load (JSON/YAML)

---

## Cell 1: Setup & Installation Check

Verify environment setup and dependencies.

In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
# Get the absolute path of the folder ONE level above this notebook
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.insert(0, root_path)

import dashboard
from dashboard.widgets import get_all_widgets, create_unified_dashboard, create_results_area
from dashboard.callbacks import setup_all_callbacks, setup_experiment_handlers

# Initialize System
widgets_dict = get_all_widgets()
setup_all_callbacks(widgets_dict)
setup_experiment_handlers(widgets_dict)  # Connects buttons to runner

print(f"✅ Dashboard v{dashboard.__version__} Loaded Successfully")

✅ Dashboard v1.1.0 Loaded Successfully


In [2]:
## Cell 2: Unified Control Panel
# Displays Tabs, Buttons, and Progress all in one view
display(create_unified_dashboard())

In [3]:
# Displays Plots and Summary Tables after experiment finishes
display(create_results_area())

### Parameter Reference Table

| Category | Parameter | Default | Range | Description |
|----------|-----------|---------|-------|-------------|
| **System** | N | 32 | 4-256 | Number of RIS elements |
| | K | 64 | 4-512 | Total probes in codebook |
| | M | 8 | 1-K | Sensing budget (probes measured) |
| | P_tx | 1.0 | 0.1-10 | Transmit power |
| | probe_type | continuous | 6 options | Probe generation method |
| **Model** | hidden_sizes | [512,256,128] | varies | Layer architecture |
| | dropout_prob | 0.1 | 0-0.8 | Dropout regularization |
| **Training** | n_train | 50000 | 1000+ | Training samples |
| | learning_rate | 1e-3 | 1e-5 to 1e-1 | Learning rate |
| | batch_size | 128 | 32-512 | Batch size |
| | n_epochs | 50 | 1-500 | Maximum epochs |
| **Eval** | top_m_values | [1,2,4,8] | 1-K | Top-m accuracy metrics |

---

## 📚 Learning Guide

### Probe Types Explained

1. **Continuous**: Random phases in [0, 2π). Best for theoretical studies.
2. **Binary**: Phases {0, π}. Simplest hardware implementation.
3. **2-bit**: Phases {0, π/2, π, 3π/2}. Good balance of performance and simplicity.
4. **Hadamard**: Structured orthogonal patterns. Excellent diversity.
5. **Sobol**: Low-discrepancy quasi-random. Better coverage than random.
6. **Halton**: Another quasi-random sequence. Similar to Sobol.

### Model Architecture Guidelines

- **Wider networks** (e.g., DoubleWide): More capacity, risk of overfitting
- **Deeper networks** (e.g., VeryDeep): Better feature extraction, harder to train
- **Pyramidal** (e.g., Pyramid): Natural information compression
- **Hourglass**: Information bottleneck for robust features
- **ResNet-style**: Same width, easier gradient flow

### Key Parameter Interactions

- **M/K ratio**: Critical for performance. Lower ratio = harder problem.
- **Learning rate**: Most important hyperparameter. Start with 1e-3.
- **Dropout**: Use 0.1-0.2 for regularization. Higher values for larger models.
- **Batch size**: Larger = more stable gradients. Smaller = better generalization.

### Typical Workflows

1. **Quick Test**: Default settings, 1 epoch, check if system works
2. **Architecture Search**: Compare multiple models, same data/training
3. **Hyperparameter Tuning**: Fix architecture, sweep learning rates
4. **Statistical Validation**: Multi-seed runs for confidence intervals
5. **Publication Results**: Best config, full training, all plots

---

## 🛠️ Troubleshooting

**Out of memory?**
- Reduce batch_size
- Use smaller model
- Reduce n_train

**Training too slow?**
- Reduce n_epochs
- Use smaller dataset
- Use simpler model

**Poor performance?**
- Increase model capacity
- Try different probe types
- Adjust learning rate
- More training data

**Overfitting?**
- Increase dropout
- Add weight_decay
- Reduce model size
- More training data

---

## 📖 References

For more information, see:
- `dashboard/README.md` - Detailed documentation
- `EXTENSION_GUIDE.md` - How to extend the system
- `USAGE_EXAMPLES.md` - Usage examples

---

**Happy Researching! 🚀**